# Applications

## Max Returns

This section will show an application of the matching algorithm when it comes
to maximizing the results through optimized assignments. 

The example below describes three machines (columns) and three operators (rows). 

[TODO: Show 2D matrix]

The matrix shows the returns if a particular operator operates a particular
machine. The goal is to find the assignment that achieves the highest return.
Only one operator is assigned to each machine and only one machine is 
assigned to each operator. 

In [1]:
from recursive import match
import numpy as np

matrix = np.array([
    [8, 4, 7],
    [5, 2, 3],
    [9, 4, 8]
])

matches = match(matrix, axis=1, limit=False)
print(f"Match: {matches}")

Match: [2 1 0]


[TODO: Explain parameters and results]

## Minimum Costs

In [ ]:
from recursive import match
import numpy as np

matrix = np.array([
    [8, 4, 7],
    [5, 2, 3],
    [9, 4, 8]
])

matrix = -1 * (matrix - np.max(matrix))

print(f"{matrix=}")

matches = match(matrix, axis=1, limit=False)
print(f"Match: {matches}")

matrix=array([[9, 9, 9],
       [9, 9, 9],
       [9, 9, 9]])


RecursionError: maximum recursion depth exceeded while calling a Python object

## Computer Vision

This section will show an application of the matching algorithm when it comes
to matching ground truth and prediction bounding boxes. 

Consider the following figure which shows a set of ground truth bounding boxes
and a set of prediction bounding boxes. 

[TODO: Show the figure]

Visually, it is clear which ground truth bounding box closely correlates 
to the prediction bounding box. 

[TODO: List the matches]

However, the IoU (intersection over union) is a metric that best describes
how well a bounding box intersects with another which is the metric used to
measure the best matches between a set of bounding boxes.

In this example, given a set of ground truth and prediction bounding boxes,
an IoU 2D matrix is generated which is taken as an input to the matching
algorithm to find the best matches for the ground truth and prediction bounding
boxes.

In [2]:
"""
Iou Methods
"""

def iou_tch(box1, box2, eps: float=1e-7):
    """
    PyTorch Batch IoU. 
    Source: https://github.com/ultralytics/yolov5
    Code: https://github.com/ultralytics/yolov5/blob/master/utils/metrics.py#L275

    Parameters
    ----------
    box1 : torch.Tensor
        The first set of bounding boxes with the shape (n, 4) 
        in the format [xmin, ymin, xmax, ymax].
    box2 : torch.Tensor
        The second bounding boxes with the shape (n, 4)
        in the format [xmin, ymin, xmax, ymax].
    eps : float
        This is used to prevent division by 0.

    Returns
    -------
    torch.Tensor
        An IoU 2D matrix which calculates
        the IoU between box1 (row) and
        box2 (column).
    """
    import torch

    (a1, a2), (b1, b2) = box1.unsqueeze(1).chunk(2, 2), box2.unsqueeze(0).chunk(2, 2)
    inter = (torch.min(a2, b2) - torch.max(a1, b1)).clamp(0).prod(2)
    return inter / ((a2 - a1).prod(2) + (b2 - b1).prod(2) - inter + eps)

def iou_npy(box1, box2, eps: float=1e-7): 
    """
    NumPy implementation of the batch IoU above. 
    NumPy is a lighter library for demonstration purposes. 

    Parameters
    ----------
    box1 : np.ndarray
        The first set of bounding boxes with the shape (n, 4) 
        in the format [xmin, ymin, xmax, ymax].
    box2 : np.ndarray
        The second bounding boxes with the shape (n, 4)
        in the format [xmin, ymin, xmax, ymax].
    eps : float
        This is used to prevent division by 0.

    Returns
    -------
    np.ndarray
        An IoU 2D matrix which calculates
        the IoU between box1 (row) and
        box2 (column).
    """
    import numpy as np

    a1, a2 = np.expand_dims(box1[:, [0,1]], 1), np.expand_dims(box1[:, [2,3]], 1)
    b1, b2 = np.expand_dims(box2[:, [0,1]], 0), np.expand_dims(box2[:, [2,3]], 0)

    inter = np.minimum(a2,b2) - np.maximum(a1,b1)
    inter = np.prod(np.clip(inter, a_min=0., a_max=np.max(inter)), axis=2)
    
    return inter / ((a2 - a1).prod(2) + (b2 - b1).prod(2) - inter + eps)

In [3]:
import numpy as np

ground_truth = np.array([[10,110,60,140], 
                         [20, 60, 50, 90], 
                         [60,40,100,100], 
                         [100,10,160,70], 
                         [20, 100, 50, 140]])
predictions = np.array([[120,100,160,140], 
                        [30,80,70,130], 
                        [70,30,90,90], 
                        [90,20,150,60], 
                        [20, 100, 50, 140]])

matrix = iou_npy(predictions, ground_truth)
print(f"IoU matrix: {matrix}")

IoU matrix: [[0.         0.         0.         0.         0.        ]
 [0.20689655 0.07407407 0.04761905 0.         0.23076923]
 [0.         0.         0.38461538 0.         0.        ]
 [0.         0.         0.04347826 0.5        0.        ]
 [0.5        0.         0.         0.         1.        ]]


The IoU matrix above is represented as the IoU values between each ground
truth against each prediction. The columns in the matrix are the ground truths
and the rows are the predictions.
Ex. ground truth at index 0 and prediction at index 1 has an IoU of 0.20689655

In [4]:
from recursive import match
matches = match(matrix, axis=1)
print(f"Matches {matches}")

Matches [ 1 -1  2  3  4]


In this example, the matches array returned is array([ 1, -1,  2,  3,  4]).
When specifying the axis to 1, we are specifying to iterate over each ground
truth (columns => axis = 1) and then find the best match of each ground truth
to the predictions.

The matches tells us that ground truth at index 0 is best matched to prediction
at index 1. Similarly, ground truth at index 2 is best matched to prediction
at index 2. The value of -1 indicates the ground truth did not match to
any prediction because the IoU was 0 for all predictions. 
The following matches can then be summarized.

GT: 0, 1,    2, 3, 4
DT: 1, null, 2, 3, 4

In [5]:
from recursive import match
matches = match(matrix, axis=0)
print(f"Matches: {matches}")

Matches: [-1  0  2  3  4]


Similarly, when setting the axis to 0, we are specifying to iterate over each detection (rows => axis = 0).

The matches tells us that detection at index 1 is best matched to ground truth 0. These results reflect
the results given when axis is set to 1. However this time, the values returned are the indices
of the best matched ground truths. The following matches can then be summarized.

DT: 0,   1, 2, 3, 4
GT: null 0, 2, 3, 4

# Benchmarks